### VGG 19 Model loading in tensorflow
Load VGG19 Model with pretrained weights.

## Weights can be downloaded by [MatConvNet.](http://www.vlfeat.org/matconvnet/pretrained/)

Importing Necessary libraries

In [1]:
import scipy.io      # For loading .mat file
import numpy as np   # for computational uses
import tensorflow as tf   # for making model


In [2]:
""" Getting File Path & then using it to access weight files"""
file=scipy.io.loadmat('./NST02/vgg19.mat')
""" Here file would contain all the data of vgg19.mat"""

' Here file would contain all the data of vgg19.mat'

In [3]:
# Checking the type & data of files ,we get
print(type(file))
print(file.keys())

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'layers', 'meta'])


In [4]:
# Our Main data would be in layers.
file=file['layers']
print(file.shape)
print(type(file))

(1, 43)
<class 'numpy.ndarray'>


In [5]:
# Above cell tells that file has 43 elements
"""Assume we want to see first element's Data"""
print(file[0][0].shape)

(1, 1)


In [6]:
# As it is having again 1 element , therefore accessing it
print(file[0][0][0][0].shape)
print(type(file[0][0][0][0]))

()
<class 'numpy.void'>


Numpy.void :Basically, these are data types where there is no pre-defined type associated
to the variable(s) you're looking at. If you look at numpy,
you have data types such as float, uint8, bool, string, etc.
void is to accommodate for more generic and flexible types and are for those 
data types that don't necessary fall into any one of these pre-defined data types.
 This situation is mostly encountered when you're loading in a struct where each element
 has multiple data types associated with multiple fields.
 Each structure element could have a combination of different data types,
 and the amalgamation of all of these data types to represent an instance of
 this structure element thus leads us to numpy.void.

In [7]:
# Further Accessing Data, we get
print(file[0][0][0][0][0])
print(file[0][0][0][0][1])
""" We get to know that 0th element contains name, first element contains type"""

['conv1_1']
['conv']


' We get to know that 0th element contains name, first element contains type'

In [8]:
print(file[0][0][0][0][2])
print(file[0][0][0][0][2].shape) # (1,2) ->  tells 2 elements , one for weight & other for Bias

""" 3rd element is again a numpy array having weights & biases for layer"""

[[array([[[[ 0.39416704, -0.08419707, -0.03631314, ..., -0.10720515,
          -0.03804016,  0.04690642],
         [ 0.46418372,  0.03355668,  0.10245045, ..., -0.06945956,
          -0.04020201,  0.04048637],
         [ 0.34119523,  0.09563112,  0.0177449 , ..., -0.11436455,
          -0.05099866, -0.00299793]],

        [[ 0.37740308, -0.07876257, -0.04775979, ..., -0.11827433,
          -0.19008617, -0.01889699],
         [ 0.41810837,  0.05260524,  0.09755926, ..., -0.09385028,
          -0.20492788, -0.0573062 ],
         [ 0.33999205,  0.13363543,  0.02129423, ..., -0.13025227,
          -0.16508926, -0.06969624]],

        [[-0.04594866, -0.11583115, -0.14462094, ..., -0.12290562,
          -0.35782176, -0.27979308],
         [-0.04806903, -0.00658076, -0.02234544, ..., -0.0878844 ,
          -0.3915486 , -0.34632796],
         [-0.04484424,  0.06471398, -0.07631404, ..., -0.12629718,
          -0.29905206, -0.2825364 ]]],


       [[[ 0.2671299 , -0.07969447,  0.05988706, ..., 

' 3rd element is again a numpy array having weights & biases for layer'

### From this we get to know that How to access weights of particular layer given Address

In [9]:
# Creates a names dictionary having names of all layers with their no
names={}
for i in range(len(file[0])):
    names[file[0][i][0][0][0][0]]=i

In [10]:
def weight(layer_name):
    """ Function that takes input of layername & returns Weights & bias of that particular layer.
    """
    layer_no=names[layer_name]    # Getting LayerNo from dictionary
    wb =file[0][layer_no][0][0][2]   # Accessing Layer Weights & Biases
    w=wb[0][0]    
    b=wb[0][1]
    name=file[0][layer_no][0][0][0]
    assert name==layer_name
    return w,b

In [11]:
def conv_relu(prev_layer,layer_no,layer_name):
    """ Function creates a Tensorflow Conv Layer getting args as prev_layer & layer_name"""
    w,b=weight(layer_name)  # Calling weight function to get weights & bias
    w=tf.constant(w)        # Create Tensorflow Constant W
    b = tf.constant(np.reshape(b, (b.size)))   # Create Tensorflow Variable b
    l=tf.nn.conv2d(prev_layer,filter=w,strides=[1,1,1,1],padding='SAME') +b 
    return tf.nn.relu(l)

In [12]:
def avg_pool(prev_layer):
    return tf.nn.avg_pool(prev_layer,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')   # Average Pooling Layer of Tensorflow

In [13]:
def model(image_height,image_width):
    graph={}
    graph['input']=tf.Variable(np.zeros((1, image_height, image_width,3)), dtype = 'float32')
    graph['conv1_1']=conv_relu(graph['input'],0,'conv1_1')
    graph['conv1_2']  = conv_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = avg_pool(graph['conv1_2'])
    graph['conv2_1']  = conv_relu(graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = conv_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = avg_pool(graph['conv2_2'])
    graph['conv3_1']  = conv_relu(graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = conv_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = conv_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = conv_relu(graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = avg_pool(graph['conv3_4'])
    graph['conv4_1']  = conv_relu(graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = conv_relu(graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = conv_relu(graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = conv_relu(graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = avg_pool(graph['conv4_4'])
    graph['conv5_1']  = conv_relu(graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = conv_relu(graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = conv_relu(graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = conv_relu(graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = avg_pool(graph['conv5_4'])
    return graph

In [14]:
graph=model(640,640)

In [15]:
graph

{'input': <tf.Variable 'Variable:0' shape=(1, 640, 640, 3) dtype=float32_ref>,
 'conv1_1': <tf.Tensor 'Relu:0' shape=(1, 640, 640, 64) dtype=float32>,
 'conv1_2': <tf.Tensor 'Relu_1:0' shape=(1, 640, 640, 64) dtype=float32>,
 'avgpool1': <tf.Tensor 'AvgPool:0' shape=(1, 320, 320, 64) dtype=float32>,
 'conv2_1': <tf.Tensor 'Relu_2:0' shape=(1, 320, 320, 128) dtype=float32>,
 'conv2_2': <tf.Tensor 'Relu_3:0' shape=(1, 320, 320, 128) dtype=float32>,
 'avgpool2': <tf.Tensor 'AvgPool_1:0' shape=(1, 160, 160, 128) dtype=float32>,
 'conv3_1': <tf.Tensor 'Relu_4:0' shape=(1, 160, 160, 256) dtype=float32>,
 'conv3_2': <tf.Tensor 'Relu_5:0' shape=(1, 160, 160, 256) dtype=float32>,
 'conv3_3': <tf.Tensor 'Relu_6:0' shape=(1, 160, 160, 256) dtype=float32>,
 'conv3_4': <tf.Tensor 'Relu_7:0' shape=(1, 160, 160, 256) dtype=float32>,
 'avgpool3': <tf.Tensor 'AvgPool_2:0' shape=(1, 80, 80, 256) dtype=float32>,
 'conv4_1': <tf.Tensor 'Relu_8:0' shape=(1, 80, 80, 512) dtype=float32>,
 'conv4_2': <tf.Tens